In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from facenet_pytorch import MTCNN, InceptionResnetV1
import os
import json
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data.dataset import Subset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
class CelebASpoofDataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        if self.split == "train":
            self.protocol_dir = "protocol1"
        else:
            self.protocol_dir = "protocol2"
        self.data = self.load_data()
        

    def load_data(self):
        data = []
        data_path = os.path.join(self.root_dir, 'Data', self.split)
        label_path = os.path.join(self.root_dir, 'metas', self.protocol_dir, f'{self.split}_label.json')

        with open(label_path, 'r') as f:
            labels = json.load(f)

        for person_id, person_data in labels.items():
            # for img_info in person_data['data']:
            #     img_path = os.path.join(data_path, person_id, img_info['image'])
            #     label = 1 if img_info['label'] == 'live' else 0  # 1 for live, 0 for spoof
            #     data.append((img_path, label))
            pass

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, label


In [13]:
class LCCFASDDataset(Dataset):
    def __init__(self, root_dir, split='training', transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.data = self.load_data()

    def load_data(self):
        data = []
        data_path = os.path.join(self.root_dir, f'LCC_FASD_{self.split}')
        labels = {'real': 1, 'spoof': 0}

        for label, label_id in labels.items():
            label_path = os.path.join(data_path, label)
            for img_name in os.listdir(label_path):
                img_path = os.path.join(label_path, img_name)
                data.append((img_path, label_id))

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, label

In [14]:
class FaceAntiSpoofingDataset(Dataset):
    def __init__(self, root_dir, celeba_split='train', lccfasd_split='training', transform=None):
        self.root_dir = root_dir
        self.transform = transform

        # Initialize CelebA_Spoof dataset
        self.celeba_dataset = CelebASpoofDataset(
            root_dir=os.path.join(self.root_dir, 'CelebA_Spoof'),
            split=celeba_split,
            transform=transform
        )

        # Initialize LCC_FASD dataset
        self.lccfasd_dataset = LCCFASDDataset(
            root_dir=os.path.join(self.root_dir, 'LCC_FASD'),
            split=lccfasd_split,
            transform=transform
        )

    def __len__(self):
        return len(self.celeba_dataset) + len(self.lccfasd_dataset)

    def __getitem__(self, idx):
        if idx < len(self.celeba_dataset):
            # Sample from CelebA_Spoof dataset
            return self.celeba_dataset[idx]
        else:
            # Sample from LCC_FASD dataset
            lccfasd_idx = idx - len(self.celeba_dataset)
            return self.lccfasd_dataset[lccfasd_idx]


In [20]:
# Usage example
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define the dataset
face_antispoofing_dataset = FaceAntiSpoofingDataset(root_dir='./datasets', celeba_split='train', lccfasd_split='training', transform=transform)

# Define the sizes of train and validation sets
train_size = int(0.8 * len(face_antispoofing_dataset))
val_size = len(face_antispoofing_dataset) - train_size

# Split the dataset
train_set, val_set = random_split(face_antispoofing_dataset, [train_size, val_size])

In [21]:
# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True, device=device)

# Initialize FaceNet model
facenet_model = InceptionResnetV1(pretrained='vggface2').eval()

In [22]:
class FaceAntiSpoofingModel(nn.Module):
    def __init__(self):
        super(FaceAntiSpoofingModel, self).__init__()

        # MTCNN for face detection and alignment
        self.mtcnn = MTCNN(keep_all=True, device=device)

        # FaceNet for feature extraction
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval()

        # Fully connected layers for classification
        self.fc1 = nn.Linear(512, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 2)  # Output has 2 classes (live or spoof)

    def forward(self, x):
        # MTCNN for face detection and alignment
        boxes, probs = self.mtcnn.detect(x)

        # Extract faces using bounding boxes
        faces = []
        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            face = x[i, :, y1:y2, x1:x2].unsqueeze(0)
            faces.append(face)

        faces = torch.cat(faces)

        # FaceNet for feature extraction
        embeddings = self.facenet(faces)

        # Fully connected layers for classification
        x = self.fc1(embeddings)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [23]:
# Define training parameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Create data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Initialize model and optimizer
model = FaceAntiSpoofingModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

    # Validation loop
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs_val, labels_val in val_loader:
            inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)

            outputs_val = model(inputs_val)
            _, predicted_val = torch.max(outputs_val.data, 1)

            total_val += labels_val.size(0)
            correct_val += (predicted_val == labels_val).sum().item()

        val_accuracy = correct_val / total_val
        print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')
# Save the trained model
torch.save(model.state_dict(), 'face_anti_spoofing_model.pth')

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
import torch
from torch.utils.data import DataLoader

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


# Initialize the model
model = FaceAntiSpoofingModel()

# Load the trained model weights
model.load_state_dict(torch.load('face_anti_spoofing_model.pth'))
model.eval()

model.to(device)

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Update counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')


In [7]:
import json

with open("../datasets/CelebA_Spoof/metas/protocol1/train_label.json", 'r') as f:
    labels = json.load(f)
labels

{'Data/train/4980/spoof/000003.jpg': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  1,
  2,
  1],
 'Data/train/1857/live/000006.jpg': [0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'Data/train/8206/spoof/000007.jpg': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  9,
  2,
  1,
  1],
 'Data/train/8032/spoof/000011.jpg': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [ ]:
os.getcwd()

'/home/deeplearner/ai-vn/face_anti_proofing/notebooks'